#  Data Load

In [1]:
import warnings

warnings.filterwarnings( 'ignore' )

In [2]:
import pandas as pd

rating_df = pd.read_csv('C:\\content_caching\\ml-25m\\ratings.csv')
rating_df

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [3]:
import time
import datetime

print('min timestamp: ',rating_df['timestamp'].min(), str(datetime.datetime.fromtimestamp(rating_df['timestamp'].min())))
print('max timestamp: ',rating_df['timestamp'].max(), str(datetime.datetime.fromtimestamp(rating_df['timestamp'].max())))

min timestamp:  789652009 1995-01-09 20:46:49
max timestamp:  1574327703 2019-11-21 18:15:03


In [4]:
standard_year = '2019'
standard_time = standard_year+'-01-01 00:00:00' # 2019년 데이터만 사용
standard_time = datetime.datetime.strptime(standard_time, '%Y-%m-%d %H:%M:%S')
standard_timestamp = int(time.mktime(standard_time.timetuple()))
print('test timestamp: ', standard_timestamp)
print('test time: ',str(datetime.datetime.fromtimestamp(standard_timestamp)))

test timestamp:  1546268400
test time:  2019-01-01 00:00:00


In [5]:
after2019_rating_df = rating_df[rating_df['timestamp'] >= standard_timestamp].reset_index(drop=True)
after2019_rating_df

,userId,movieId,rating,timestamp
0,3,1274,4.5,1566088819
1,3,1732,4.5,1566091883
2,3,1960,4.0,1566090766
3,3,2021,4.0,1566089449
4,3,3408,3.5,1566089773
...,...,...,...,...
1202597,162536,148626,5.0,1572259347
1202598,162536,148685,5.0,1572257813
1202599,162536,164179,4.0,1572259078
1202600,162536,202429,4.0,1572257782


In [6]:
print('number of  movieIds : ', after2019_rating_df['movieId'].nunique())
print('number of  userId : ', after2019_rating_df['userId'].nunique())

number of  movieIds :  41440
number of  userId :  10619


In [7]:
standards = {'January':['2019-01-01 00:00:00', '2019-02-01 00:00:00'],
             'February':['2019-02-01 00:00:00', '2019-03-01 00:00:00'],
             'March':['2019-03-01 00:00:00', '2019-04-01 00:00:00'],
             'April':['2019-04-01 00:00:00', '2019-05-01 00:00:00'],
             'May':['2019-05-01 00:00:00', '2019-06-01 00:00:00'],
             'June':['2019-06-01 00:00:00', '2019-07-01 00:00:00'],
             'July':['2019-07-01 00:00:00', '2019-08-01 00:00:00'],
             'August':['2019-08-01 00:00:00', '2019-09-01 00:00:00'],
             'September':['2019-09-01 00:00:00', '2019-10-01 00:00:00'],
             'October':['2019-10-01 00:00:00', '2019-11-01 00:00:00'],
             'November':['2019-11-01 00:00:00', '2019-12-01 00:00:00'],
             'December':['2019-12-01 00:00:00', '2020-01-01 00:00:00']
             }

Month = list(standards.keys())[12-1] # 4월 -1 index
print(Month)

start_time = datetime.datetime.strptime(standards[Month][0], '%Y-%m-%d %H:%M:%S')
end_time = datetime.datetime.strptime(standards[Month][1], '%Y-%m-%d %H:%M:%S')

start_timestamp = int(time.mktime(start_time.timetuple()))
end_timestamp = int(time.mktime(end_time.timetuple()))

print(start_time, start_timestamp)
print(end_time, end_timestamp)

month_rating_df = after2019_rating_df[(after2019_rating_df['timestamp'] < end_timestamp) &
                                   (after2019_rating_df['timestamp'] >= start_timestamp)].reset_index(drop=True)

# 10월까지 1,216 개 생성. 
# 11월까지 1,300 개 생성. 
if Month == 'November': # 12월 데이터는 존재하지 않음. 11월 존재하는 데이터까지만 진행.
    end_timestamp = month_rating_df['timestamp'].max()

month_rating_df

December
2019-12-01 00:00:00 1575126000
2020-01-01 00:00:00 1577804400


,userId,movieId,rating,timestamp


In [8]:
# 해당 월에 데이터가 없다면 종료.
import sys

if len(month_rating_df) == 0:
    sys.exit('해당 월에 데이터 없음')

SystemExit: 해당 월에 데이터 없음

In [ ]:
print('number of  movieIds : ', month_rating_df['movieId'].nunique())
print('number of  userId : ', month_rating_df['userId'].nunique())

In [ ]:
month_rating_df['movieId'].value_counts()

In [ ]:
import pandas as pd

# Assuming your DataFrame is named month_rating_df

# Count the frequency of each movieId
movie_frequency = month_rating_df['movieId'].value_counts()

# Create a dictionary to map original movieIds to their corresponding frequencies
movie_frequency_map = movie_frequency.to_dict()

# Sort the dictionary by frequency and assign new movieId values starting from 0
sorted_movie_ids = sorted(movie_frequency_map, key=lambda x: movie_frequency_map[x], reverse=True)
remapped_ids = {movie_id: idx for idx, movie_id in enumerate(sorted_movie_ids)}

# Apply the remapping to the DataFrame
month_rating_df['remapped_movieId'] = month_rating_df['movieId'].map(remapped_ids)

# Now month_rating_df contains a new column 'remapped_movieId' with the remapped values
month_rating_df

In [ ]:
month_rating_df['remapped_movieId'].value_counts()

In [ ]:
movie_id_probabilites = month_rating_df['remapped_movieId'].value_counts() / month_rating_df['remapped_movieId'].value_counts().sum()
movie_id_probabilites

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Visualize the probabilities for the first row
plt.figure(figsize=(10, 6))
plt.plot(range(len(movie_id_probabilites.values)), movie_id_probabilites.values, marker='o')
plt.xlabel('number of request ascending order')
plt.ylabel('Probability')
plt.title('Probabilities for request frequency')
plt.grid(True)
plt.show()

In [ ]:
movie_id_probabilites.sum()

In [ ]:
pareto = int(month_rating_df['movieId'].nunique() * 0.2)
print('top 20% 개수 : ', pareto)

pareto_probability = sum(movie_id_probabilites.values[:pareto+1])
print('top 20% 비율 : ', pareto_probability)

In [ ]:
number_of_service_Ids = 64

Y = 0.2

pareto_number_of_service_ids = int(number_of_service_Ids * Y)

pareto_number_of_movies_per_service = int(pareto / pareto_number_of_service_ids) + 1
print('pareto : ',pareto_number_of_service_ids, pareto_number_of_movies_per_service)

not_pareto_number_of_service_ids = number_of_service_Ids - pareto_number_of_service_ids
try:
    not_pareto_number_of_movies_per_service = int((month_rating_df['movieId'].nunique()-pareto) / not_pareto_number_of_service_ids) + 1
except:
    not_pareto_number_of_movies_per_service = 0
print('rest pareto : ', not_pareto_number_of_service_ids, not_pareto_number_of_movies_per_service)

In [ ]:
# Create a mapping dictionary for remapped_movieId to service_id
mapping_dict = {}

pareto = pareto_number_of_service_ids * pareto_number_of_movies_per_service

# Map the remapped_movieId values to service_id values
current_service_id = 0
for value in range(pareto):
    if value % pareto_number_of_movies_per_service == 0 and value != 0:
        current_service_id += 1
    mapping_dict[value] = current_service_id

for value in range(pareto, month_rating_df['movieId'].nunique()):
    tmp_value = value-pareto
    try:
        if tmp_value % not_pareto_number_of_movies_per_service == 0:
            current_service_id += 1
        mapping_dict[value] = current_service_id
    except:
        break

# 실험단계
매 time slot 시작될 때 마다
1. movie  묶음  ID와  각 mec별  service Id mapping을  random하게 변경
2. 무비랜즈 파일에서 t-1 시점 마지막 movie를 선택한 user id가 u라면 t시점 첫번째 movie를 선택한 user id는 (u+1)%user수
3. 선택한무비가위에서나눈128개의서비스ID중어디에속하는지판단하여해당서비스ID의요청횟수1증가시킴
4. 이와같은방법으로timeslott동안서비스ID별요청횟수를얻음

In [ ]:
import random

# Set the seed value
random.seed(42)

# Example usage
number_of_sample_users = 1000
number_of_mecs = 9  # Number of sqrt MECs
mecs = [(x, y) for x in range(int(number_of_mecs**(1/2))) for y in range(int(number_of_mecs**(1/2)))]

print(mecs)

mean_velocity = 1.0
correlation_time = 0.5

# Number of move
num_steps = 1

now_positions = [(round(random.uniform(0, int(number_of_mecs**(1/2))-1), 13), round(random.uniform(0, int(number_of_mecs**(1/2))-1), 13)) for _ in range(number_of_sample_users)]
print('min,  max : ', 0, int(number_of_mecs**(1/2))-1)
print('now_positions[1] : ', now_positions[10])

In [ ]:
data = {
    'userId' : range(number_of_sample_users),
    'x_position' : [now_position[0] for now_position in now_positions],
    'y_position' : [now_position[1] for now_position in now_positions]
}

user_df = pd.DataFrame(data)
user_df

In [ ]:
import numpy as np

mecs_array = np.array(mecs)

def calculate_nearest_mec(group):
    distances = np.sqrt((group['x_position'].values[:, np.newaxis] - mecs_array[:, 0])**2 + (group['y_position'].values[:, np.newaxis] - mecs_array[:, 1])**2)
    nearest_mec_index = np.argmin(distances)
    group['nearest_mec'] = nearest_mec_index
    return group

user_df = user_df.groupby('userId').apply(calculate_nearest_mec)
user_df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Replace 'specific_column' and 'other_column' with the actual column names you want to visualize
plt.scatter(user_df['x_position'],user_df['y_position'])
plt.xlabel('x_position')
plt.ylabel('y_position')
plt.title('Scatter Plot')
plt.show()

In [ ]:
# time_t1, time_interval 모두 timestamp 형태

import pandas as pd
import numpy as np

def make_request_list(dataframe, now, time_interval):
    now_datetime = pd.to_datetime(now, unit='s')
    end_datetime = now_datetime + pd.Timedelta(hours=time_interval)
    request_df = dataframe[(pd.to_datetime(dataframe['timestamp'], unit='s') >= now_datetime) & (pd.to_datetime(dataframe['timestamp'], unit='s') <= end_datetime)]
    end_time = int(end_datetime.timestamp())
    return request_df, end_time

now = start_timestamp  # 현재 시간(timestamp)
time_interval = 6  # 시간 간격(hours 단위)
ex_rating_df = month_rating_df.copy()

In [ ]:
# Function to map request_movie_ids based on nearest_mec
def map_movie_ids_to_service_ids(row):
    nearest_mec = row['nearest_mec']
    request_movie_ids = row['request_movie_ids']

    if 0 <= nearest_mec < len(service_id_per_mec):
        service_dict = service_id_per_mec[nearest_mec]

        request_service_ids = []
        for request_movie_id in request_movie_ids:
            try: # Y=1 인 경우 하위 80프로 movieId는 할당되는 service가 없음 -> except문 처리
                request_service_id = service_dict[request_movie_id]
                request_service_ids.append(request_service_id)
            except:
                continue

    return request_service_ids

In [ ]:
import os

save_path = "./heatmap"+'_case3_'+str(Y)

try:
    os.mkdir(save_path)
except:
    print('[Errno 17] File exists')

In [ ]:
import os

num=0

for filename in os.listdir(save_path):
    if filename.lower().endswith('.png'):
        num+=1
num

In [ ]:
import pandas as pd
import random
from PIL import Image

random.seed(42)

end_time = now
now_user_id = 0

# Determine the maximum values for creating the matrix
max_nearest_mec = number_of_mecs
max_request_service_id = number_of_service_Ids

while end_time < end_timestamp:
    print('save image index:',num)

    request_df, end_time = make_request_list(ex_rating_df, end_time, time_interval)

    # Create a dictionary to map user IDs to their corresponding service_ids
    request_df['userId'] = [(now_user_id+i)%number_of_sample_users for i in range(len(request_df))]
    try:
        now_user_id = int(request_df.iloc[-1]['userId'])+1 # user+1반영
    except:
        print('no request time')
    user_movie_mapping = request_df.groupby('userId')['remapped_movieId'].apply(list).to_dict()

    # Shuffle(=random) movie_set_id to service_id mapping

    # Step 1: Create empty groups
    groups = [[] for _ in range(64)]

    # Step 2: Populate groups based on values
    for key, value in mapping_dict.items():
        groups[value].append((key))

    # Step 3: Update service_id_per_mec
    service_id_per_mec = []
    for mec in range(number_of_mecs):
        random.shuffle(groups)

        mapping_dict = {}
        for service_id, keys in enumerate(groups):
            for key in keys:
                mapping_dict[key] = service_id

        service_id_per_mec.append(mapping_dict)

    #print(service_id_per_mec[0])
    user_df['request_movie_ids'] = user_df['userId'].map(user_movie_mapping)
    user_df['request_movie_ids'] = user_df['request_movie_ids'].fillna("").apply(list)


    # Apply the mapping function to the DataFrame
    user_df['request_service_ids'] = user_df.apply(map_movie_ids_to_service_ids, axis=1)

    # Initialize the matrix with zeros
    matrix = np.zeros((max_nearest_mec, max_request_service_id), dtype=int)

    # Populate the matrix based on the DataFrame
    for _, row in user_df.iterrows():
        nearest_mec = row['nearest_mec']
        request_service_ids = row['request_service_ids']

        for service_id in request_service_ids:
            matrix[nearest_mec][service_id] += 1

    normalized_matrix = matrix / matrix.max(axis=1, keepdims=True)

    # Create a PIL image from the array
    image = Image.fromarray((255 * normalized_matrix).astype(np.uint8), mode='L')

    file_name = '/train'+str(num)+'.png'
    image.save(save_path+file_name)

    num+=1

# 생성 확인

In [ ]:
from IPython.display import display
from PIL import Image

# Set up the path to the saved image
save_path = "./heatmap"+'_case2_'+str(Y)
file_name = '/train'+str(num-1)+'.png'


# Load the image
image = Image.open(save_path+file_name)

# Display the image
display(image)

In [ ]:
image_array = np.array(image)
image_array.shape